# Using the ASE VASP Calculator
Here I will show you how to use the built in Vasp calculator with ASE. There is a little bit of set up required.
## Step 0:
Install ASE version 3.21.1 with:
~~~
pip install --user --upgrade ase==3.21.1
~~~

## Step 1:
Put the run_vasp.py somewhere in your work space.

In the run_vasp.py file you can change two variables depending on how you want to submit jobs.
~~~
q = '*@@schneider_q16copt'
~~~
* change this to the queue you want to submit to

~~~
pe = 'mpi-64 64'
~~~
* change this to the parallel enviornment you want to use



## Step 2:
Add the following lines to your .bashrc:
~~~
export VASP_SCRIPT=~/path/to/run_vasp.py
export VASP_PP_PATH=~/path/to/POTCARs
~~~
## Step 3:
Here we have to modify the ase code a little bit. If you update ase these changes will go away. Luckily I have this reference sheet for you, so you can go back and fix it.

### Edit vasp Calculator Files
The location of these files will depend on how you installed ASE, but if you installed it the way the website recomends the files will be in:
~~~
~/.local/lib/python3.x/site-packages/ase
~~~
* ase/calculators/vasp/creat_input.py
  * line 1070: change 'lda' to 'pbe'
  * the original line states:
  ~~~
  p.update({'pp': 'lda'})
  ~~~

* ase/calculators/vasp/vasp.py
  * comment out lines 330 & 331
  * these lines state:
  ~~~
  self.update_atoms(atoms)
  self.read_results()
  ~~~

# Running Jobs
So you set up ase, you made the recommended modifcations, and now you are ready to calculate things.

We will start with running new jobs using ase. 

In [2]:
from ase import Atoms,Atom
from ase.io import read,write
from ase.calculators.vasp import Vasp

In [2]:
# build a water molecule
atoms = Atoms([Atom('H', [0.5960812, -0.7677068, 0.000]),
               Atom('O', [0.00000, 0.00000, 0.00000]),
               Atom('H', [0.5960812, 0.76776068, 0.00000])],
               cell = (8,8,8),
             pbc = [1,1,1])
atoms.center()

In [8]:
# set up a calculator, i'm doing a simple geometry optimization
calc = Vasp(directory ='h2o',
            encut=400,
            xc='pbe',
            ismear = 0,
            ibrion =1,
            ediff = 1e-4,
            ncore =64,
            ediffg = -0.03,
            nsw = 500)


In [9]:
# run that calculator on your atoms
calc.calculate(atoms)

# Wait
Now we need to wait for this calculation to run. Go get a cup of coffee, read a paper, check on your dog. 

Once the calculation is done, you can get the results by doing the following:

In [3]:
calc = Vasp(directory='h2o')
calc.read()
atoms = calc.get_atoms()
print('Converged? ', calc.converged)
print(atoms.get_potential_energy())
print(atoms.get_forces())

-14.22667867
[[-0.00174262 -0.00041778  0.        ]
 [ 0.00348524  0.          0.        ]
 [-0.00174262  0.00041778  0.        ]]


# What About Jobs Ran Before We Used ASE
Don't worry, we can load them with this calculator now and get the results. 

In [5]:
# let's load an old job 
calc = Vasp(directory='afv_test')

In [6]:
# the difference here is we need to tell ase to read the results
calc.read()
atoms=calc.get_atoms()
print(atoms.get_potential_energy())

-711.60269729


# Other Useful Commands
Here we will use our water calculation to provide some examples of things ASE Vasp can do.

In [3]:
calc = Vasp(directory='h2o')
calc.read()
atoms = calc.get_atoms()

## Two ways to get data:

In [5]:
forces = atoms.get_forces()
print(forces[0])
forces = calc.get_forces()
print(forces[0])

[-0.00174262 -0.00041778  0.        ]
[-0.00174262 -0.00041778  0.        ]


In [7]:
energy = atoms.get_potential_energy()
print(energy)
energy = calc.get_potential_energy()
print(energy)

-14.22667867
-14.22667867


## What else can we get?

In [12]:
# Check convergence and number of ionic steps:

print('Converged: ',calc.converged)
print('Ionic Updates: ',calc.read_number_of_ionic_steps())

Converged:  True
Ionic Updates:  3


In [18]:
# How to get stress... (go to grad school)

print('Stress: \n',calc.get_stress()[:3])

Stress: 
 [0.00154229 0.00134626 0.00164071]


In [20]:
# Number of electrons:

print('# of electrons: ',calc.get_number_of_electrons())

# of electrons:  8.0


In [25]:
# Number of bands and occupation:

print('# of bands: ', calc.get_number_of_bands())
print('Occupations numbers: ',calc.get_occupation_numbers())

# of bands:  8
Occupations numbers:  [2. 2. 2. 2. 0. 0. 0. 0.]


In [22]:
# Number of spins:

print('# of spins: ', calc.get_number_of_spins())

# of spins:  1


# Calc as a dictionary
This will read all the information about the calculation and store it as a dictionary.

In [31]:
d = calc.asdict()

In [32]:
# Let's see what items are stored in that dinctionary:

for x in d:
    print(x)

ase_version
vasp_version
inputs
results
atoms


In [35]:
print(d['ase_version'])
print(d['vasp_version'])

3.21.1
5.4.1


In [38]:
inputs = d['inputs']
for x in inputs:
    print(x,inputs[x])

encut 400.0
ediff 0.0001
ediffg -0.03
gga PE
ibrion 1
ismear 0
nsw 500
ncore 64
pp PBE
txt -
kpts [1 1 1]
gamma False
reciprocal False
ignore_constraints False
custom {}
